In [3]:
import pandas as pd
import numpy as np

training = pd.read_csv('new train data.csv') #QUESTION: where did we get this csv from? column values are similar to train_transaction but w/ less
# pd.set_option('display.max_columns', 150)
# pd.set_option('display.max_rows', 150)

# ProductCD: product code of each transaction; there are 5 product types
training['ProductCD'].value_counts()

W    24264
C    10054
R     2719
H     2651
S     1100
Name: ProductCD, dtype: int64

In [4]:
training.columns.values

array(['Unnamed: 0', 'TransactionID', 'isFraud', 'TransactionDT',
       'TransactionAmt', 'ProductCD', 'card1', 'card3', 'card4', 'card5',
       'card6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
       'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'V95', 'V96', 'V97',
       'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105',
       'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113',
       'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121',
       'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129',
       'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137',
       'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286',
       'V287', 'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294',
       'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V302',
       'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310',
       'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318',
       'V319'

In [6]:
# card4: brand of card; there are 4 brands
training['card4'].value_counts()

visa                26710
mastercard          12944
discover              609
american express      525
Name: card4, dtype: int64

In [8]:
# card6 contains types of cards
training['card6'].value_counts()

debit              26016
credit             14769
debit or credit        2
charge card            1
Name: card6, dtype: int64

In [18]:
# Drop the 3 unique rows of 'debit or credit' and 'charge card'
training['card6'].value_counts()
mask = training['card6'] != 'debit or credit'
training = training[mask]
mask2 = training['card6'] != 'charge card'
training = training[mask2]
training.shape

(40785, 112)

In [27]:
# create into category type
training['card6'] = training['card6'].astype('category')
training['card4'] = training['card4'].astype('category')
training['ProductCD'] = training['ProductCD'].astype('category')

# encode into numeric categories; we opt for dummy variables (One Hot Encoding)
training_dummies = pd.get_dummies(training, columns=['card4', 'card6', 'ProductCD'])

In [85]:
from sklearn import preprocessing

# initialize scaler
scaler = preprocessing.MinMaxScaler()

# drop target column
training_v1 = training_dummies.drop('TransactionID', axis=1)
training_v2 = training_v1.drop('isFraud', axis=1)
training_y = training_dummies['isFraud'].copy()
training_id = training_dummies['TransactionID'].copy()

# Fit the data with scaler (scales data)
# looper = [x for x in training_v2.columns]
# training_std[looper] = ss.fit_transform(training_v2[looper])
training_v2 = training_v2.set_index('Unnamed: 0')
x = training_v2.values
x_scaled = scaler.fit_transform(x)
training_scaled = pd.DataFrame(x_scaled)
training_scaled = training_scaled.set_axis([x for x in training_v2.columns], axis=1, inplace=False)
training_scaled.insert(loc=0, column='TransactionID', value=training_id.map(lambda x: int(x)))
training_scaled.head()

,TransactionID,TransactionDT,TransactionAmt,card1,card3,card5,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_credit,card6_debit,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W
0,3273604.0,0.443036,0.020744,0.488205,0.381679,0.919708,0.000214,0.000176,0.0,0.000000,0.003021,0.000444,0.0,0.0000,0.004926,0.000000,0.000314,0.0,0.000686,0.000701,0.015649,0.0,0.000759,0.0,0.0,0.013333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,0.142857,0.142857,0.111111,0.111111,0.111111,0.166667,0.166667,0.166667,0.5,0.5,0.5,0.333333,0.333333,0.333333,0.076923,0.076923,0.076923,0.0,0.000786,0.0,0.0,0.003569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0625,0.014706,0.0,0.010638,0.0,0.000000,0.25,0.142857,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000788,0.000000,0.0,0.003569,0.0,0.000000,0.02595,0.020825,0.024043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2999170.0,0.016551,0.011435,0.443326,0.381679,0.919708,0.013456,0.007557,0.0,0.000000,0.036254,0.018214,0.0,0.0000,0.187192,0.000000,0.013183,0.0,0.049743,0.025947,0.738654,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,0.142857,0.142857,0.111111,0.111111,0.111111,0.166667,0.166667,0.166667,0.5,0.5,0.5,0.333333,0.333333,0.333333,0.076923,0.076923,0.076923,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001087,0.0,0.0000,0.000000,0.0,0.021277,0.0,0.037037,0.00,0.000000,0.0,0.03125,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000801,0.000606,0.0,0.003624,0.0,0.006596,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,3023872.0,0.053123,0.014002,0.918124,0.381679,0.919708,0.023708,0.016872,0.0,0.000000,0.395770,0.032874,0.0,0.0000,0.315271,0.000000,0.024168,0.0,0.133105,0.057504,0.093897,0.0,0.003037,0.0,0.0,0.053333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,0.142857,0.142857,0.111111,0.111111,0.111111,0.166667,0.166667,0.166667,0.5,0.5,0.5,0.333333,0.333333,0.333333,0.076923,0.076923,0.076923,0.0,0.004832,0.0,0.0,0.021950,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.042553,0.0,0.000000,0.00,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.004849,0.000000,0.0,0.021950,0.0,0.000000,0.00000,0.005787,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,3033043.0,0.064770,0.009580,0.663003,0.381679,0.905109,0.000214,0.000176,0.0,0.000444,0.000000,0.000444,0.0,0.0003,0.000000,0.000307,0.000314,0.0,0.000343,0.000701,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,0.142857,0.142857,0.111111,0.111111,0.111111,0.166667,0.166667,0.166667,0.5,0.5,0.5,0.333333,0.333333,0.333333,0.076923,0.076923,0.076923,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0625,0.014706,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.090909,0.111111,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,3422561.0,0.695863,0.025946,0.488205,0.381679,0.919708,0.000641,0.000351,0.0,0.000000,0.003021,0.000444,0.0,0.0000,0.009852,0.000000,0.000628,0.0,0.001372,0.002104,0.000000,0.0,0.000759,0.0,0.0,0.013333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.142857,0.142857,0.142857,0.111111,0.111111,0.111111,0.166667,0.166667,0.166667,0.5,0.5,0.5,0.